# Homework 4 
Name: Siddharth Iyer <br>
CWID: 10447455

## Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

## Problem 1

**(a) Is such an option cheaper or more expensive than a vanilla put option? Explain.**

A barrier put option should be cheaper than a vanilla put option because there is an additional condition that increases the likelihood of expiring worthless. A vanilla put option will have fewer 0s at expiry in its Monte Carlo simulation, so naturally you would pay more when it is discounted.

**(b) Is the estimator $\hat{P_{n,m}}$ biased? Is it biased low or high? Explain.**

The estimator is biased high. Because the estimation relies on discrete GBM simulation, which tends to under-represent stock price due to discrete holding error, it will be over-represent the put option value. 

**(c) Use the parameters in the table below to compute an estimate of the price using (0.1) along with a 95% confidence interval. Use m = 63 and n equal to at least 100,000.**

In [2]:
nsims = 100000
discrete_freq = 252
S0 = 50
sig = .3
r = .05
K = 60
T = .25
H = 55

In [3]:
# simulates GBM paths
def simulate_gbm_paths(nsims, S0, sig, mu, T, discrete_freq):
    '''
    nsims: number of simulations to produce
    S0: initial stock price
    sig: volatility expressed as a percentage
    mu: annualized drift expressed as a percentage
    r: interest rate
    T: time in years
    discrete_freq: number of discrete time intervals per increment of T (252 would be 1 trading year)
    '''
    nparts = int(T)*discrete_freq + int(discrete_freq * (T - np.floor(T)))
    dt = T / nparts
    Xt = np.log(S0) + np.cumsum(( (mu - sig**2/2)*dt + sig*np.sqrt(dt) * np.random.normal(size=(nparts,nsims)) ), axis=0)
    Xt = np.exp(Xt)
    Xt = np.vstack([np.repeat(S0, nsims), Xt])

    return Xt

def barrier_option_pricing_monte_carlo(nsims, S0, sig, r, T, discrete_freq, H, optype = "uop"):
    gbm_sim = simulate_gbm_paths(nsims, S0, sig, r, T, discrete_freq)
    max_array = np.max(gbm_sim, axis=0)
    if optype == "uop":            
        return np.exp(-r*T) * np.sum((np.maximum(K - gbm_sim[-1], 0) * (max_array < H)*1)) * 1/nsims


In [4]:
gbm_sim = simulate_gbm_paths(nsims, S0, sig, r, T, discrete_freq)

In [5]:
fig = px.line(gbm_sim[:, 0:100], width=800, title = "GBM Simulation")
fig.show()
fig = px.histogram(np.max(gbm_sim, axis=0), width=800, nbins = 150, title="Maximum Value of Stock Process")
fig.show()

C:\Users\sidiy\AppData\Local\Programs\Python\Python38\lib\site-packages\plotly\express\_core.py:1222: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_output[col_name] = to_unindexed_series(df_input[argument])


In [191]:
barrier_option_pricing_monte_carlo(nsims, S0, sig, r, T, discrete_freq, H, optype="uop")

7.352723976215193